In [45]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import itertools
from PIL import Image 
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dropout,Flatten,Dense,Activation,BatchNormalization,Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from  sklearn.metrics import confusion_matrix

In [46]:
df = pd.DataFrame(columns = ['image_path','0label'])
labels = {
    r"C:\Users\NICE\Downloads\archive (1)\data\water" :'water',
    r"C:\Users\NICE\Downloads\archive (1)\data\cloudy": 'cloudy',
    r"C:\Users\NICE\Downloads\archive (1)\data\green_area" : 'green_area',
    r"C:\Users\NICE\Downloads\archive (1)\data\desert" : 'desert'
}

In [47]:
import os
import pandas as pd

  
df = pd.DataFrame(columns=['image_path', 'label'])

for folder in labels:
    if not os.path.exists(folder):
        print(f"Folder '{folder}' does not exist.")
        continue
    for image_name in os.listdir(folder):
        image_path = os.path.join(folder, image_name)
        if os.path.isfile(image_path):
            label = labels[folder]
            df = pd.concat([df, pd.DataFrame({'image_path': [image_path], 'label': [label]})], ignore_index=True)

print(df)


                                             image_path   label
0     C:\Users\NICE\Downloads\archive (1)\data\water...   water
1     C:\Users\NICE\Downloads\archive (1)\data\water...   water
2     C:\Users\NICE\Downloads\archive (1)\data\water...   water
3     C:\Users\NICE\Downloads\archive (1)\data\water...   water
4     C:\Users\NICE\Downloads\archive (1)\data\water...   water
...                                                 ...     ...
5626  C:\Users\NICE\Downloads\archive (1)\data\deser...  desert
5627  C:\Users\NICE\Downloads\archive (1)\data\deser...  desert
5628  C:\Users\NICE\Downloads\archive (1)\data\deser...  desert
5629  C:\Users\NICE\Downloads\archive (1)\data\deser...  desert
5630  C:\Users\NICE\Downloads\archive (1)\data\deser...  desert

[5631 rows x 2 columns]


In [48]:
df.to_csv("Image_data_set.csv",index =False)

In [49]:
df = pd.read_csv("Image_data_set.csv")
df

,image_path,label
0,C:\Users\NICE\Downloads\archive (1)\data\water...,water
1,C:\Users\NICE\Downloads\archive (1)\data\water...,water
2,C:\Users\NICE\Downloads\archive (1)\data\water...,water
3,C:\Users\NICE\Downloads\archive (1)\data\water...,water
4,C:\Users\NICE\Downloads\archive (1)\data\water...,water
...,...,...
5626,C:\Users\NICE\Downloads\archive (1)\data\deser...,desert
5627,C:\Users\NICE\Downloads\archive (1)\data\deser...,desert
5628,C:\Users\NICE\Downloads\archive (1)\data\deser...,desert
5629,C:\Users\NICE\Downloads\archive (1)\data\deser...,desert


In [50]:
# Split data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Print results
print("Training Data:\n", train_df)
print("\nTesting Data:\n", test_df)

Training Data:
                                              image_path       label
3209  C:\Users\NICE\Downloads\archive (1)\data\green...  green_area
2077  C:\Users\NICE\Downloads\archive (1)\data\cloud...      cloudy
2131  C:\Users\NICE\Downloads\archive (1)\data\cloud...      cloudy
2254  C:\Users\NICE\Downloads\archive (1)\data\cloud...      cloudy
4998  C:\Users\NICE\Downloads\archive (1)\data\deser...      desert
...                                                 ...         ...
3772  C:\Users\NICE\Downloads\archive (1)\data\green...  green_area
5191  C:\Users\NICE\Downloads\archive (1)\data\deser...      desert
5226  C:\Users\NICE\Downloads\archive (1)\data\deser...      desert
5390  C:\Users\NICE\Downloads\archive (1)\data\deser...      desert
860   C:\Users\NICE\Downloads\archive (1)\data\water...       water

[4504 rows x 2 columns]

Testing Data:
                                              image_path       label
3135  C:\Users\NICE\Downloads\archive (1)\data\green...  gr

In [51]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest'  # Ensure this is placed properly
)


In [52]:
train_datagen = ImageDataGenerator( rescale=1./255)

In [53]:


# Flow data from dataframe
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_path',
    y_col='label',
    target_size=(64,64),
    batch_size=32,
    class_mode='categorical'  # Change to 'categorical' for multi-class classification
)

test_generator = train_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='image_path',
    y_col='label',
    target_size=(64,64),
    batch_size=32,
    class_mode='categorical'
)




Found 4504 validated image filenames belonging to 4 classes.
Found 1127 validated image filenames belonging to 4 classes.


In [54]:
# Print sample batch shape
batch_images, batch_labels = next(train_generator)
print(f"Batch images shape: {batch_images.shape}, Batch labels shape: {batch_labels.shape}")

Batch images shape: (32, 64, 64, 3), Batch labels shape: (32, 4)


In [56]:

model = Sequential()
model.add(Conv2D(32, (3,3), input_shape=(255,255,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)                    │ (None, 253, 253, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 126, 126, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 124, 124, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 246016)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 128)                 │      31,490,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 4)                   │             260 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 31,518,084 (120.23 MB)

 Trainable params: 31,518,084 (120.23 MB)

 Non-trainable params: 0 (0.00 B)